In [11]:
import pandas as pd
import numpy as np
import csv
pd.options.display.max_columns=50

In [ ]:
import pymongo
from pymongo import MongoClient

In [ ]:
client = MongoClient('mongodb://localhost:27017')

In [ ]:
db=client["dev_comexstat_api2"]

In [ ]:
coll_tab02=db["depara_ncm_secao_classe"]
coll_tab03=db["depara_pais"]
coll_tab04=db["depara_sh4"]
coll_tab06=db["depara_urf_lat_long"]
coll_tab07=db["depara_via"]
coll_tab08=db["depara_unid"]

In [ ]:
pipe_tab02 = [{'$project':{'_id':0, 
                           'CO_NCM': 1,
                           'NO_NCM_POR': 1,
                           'NO_ISIC_SECAO': 1,
                           'NO_ISIC_CLASSE':1 }}]

pipe_tab03 = [{'$project':{'_id':0,
                           'CO_PAIS': 1,
                           'NO_PAIS': 1 }}]

pipe_tab04 = [{'$project':{'_id':0,
                           'CO_SH4': 1,
                           'NO_SH4_POR':1 }}]

pipe_tab06 = [{'$project':{'_id':0, 
                           'CO_URF': 1,
                           'NO_URF2': 1,
                           'LAT': 1,
                           'LONG': 1 }}]

pipe_tab07 = [{'$project':{'_id':0, 
                           'CO_VIA': 1,
                           'NO_VIA': 1 }}]

pipe_tab08 = [{'$project':{'_id':0,
                           'CO_UNID': 1,
                           'SG_UNID':1 }}]


In [ ]:
cursor_tab02=coll_tab02.aggregate(pipe_tab02)
cursor_tab03=coll_tab03.aggregate(pipe_tab03)
cursor_tab04=coll_tab04.aggregate(pipe_tab04)
cursor_tab06=coll_tab06.aggregate(pipe_tab06)
cursor_tab07=coll_tab07.aggregate(pipe_tab07)
cursor_tab08=coll_tab08.aggregate(pipe_tab08)

In [ ]:
list_tab02=list(cursor_tab02)
list_tab03=list(cursor_tab03)
list_tab04=list(cursor_tab04)
list_tab06=list(cursor_tab06)
list_tab07=list(cursor_tab07)
list_tab08=list(cursor_tab08)

In [ ]:
df_tab02=pd.DataFrame(list_tab02)
df_tab03=pd.DataFrame(list_tab03)
df_tab04=pd.DataFrame(list_tab04)
df_tab06=pd.DataFrame(list_tab06)
df_tab07=pd.DataFrame(list_tab07)
df_tab08=pd.DataFrame(list_tab08)

In [ ]:
base_imp_NCM=r"C:\Users\jetin\Documents\FATEC\2° semestre\api2\Dados\IMP_2023.csv"
base_exp_NCM=r"C:\Users\jetin\Documents\FATEC\2° semestre\api2\Dados\EXP_2023.csv"

In [ ]:
dict_type={'CO_ANO':'category',
           'CO_MES':'category',
           'CO_NCM':'category',
           'CO_UNID':'category',
           'CO_PAIS':'category',
           'SG_UF_NCM':'category',
           'CO_VIA':'category',
           'CO_URF':'category',
           'SH4':'category',
           'CO_MUN':'category' }

In [ ]:
df_imp_NCM=pd.read_csv(base_imp_NCM, sep=";", dtype=dict_type)
df_exp_NCM=pd.read_csv(base_exp_NCM, sep=";", dtype=dict_type)

In [ ]:
df_exp_NCM.info()

### BASE IMPORTAÇÃO

In [ ]:
#inserindo cod_sh4 pelo cod_ncm
df_imp_NCM['CO_SH4']=[numero[0:4] for numero in df_imp_NCM['CO_NCM']]

In [ ]:
df_imp_NCM.head(2)

In [ ]:
#procurando nulos no depara sh4
df_tab04=df_tab04.loc[df_tab04['CO_SH4'].notnull()]

In [ ]:
#contando numeros cod_sh4
df_tab04.duplicated(subset=['CO_SH4']).sum()

In [ ]:
df_merge1=pd.merge(df_imp_NCM, df_tab06, how='left', on='CO_URF')

In [ ]:
df_merge2=pd.merge(df_merge1, df_tab04, how='left', on='CO_SH4')

In [ ]:
df_merge2.head(2)

In [ ]:
df_merge3=pd.merge(df_merge2, df_tab03, how='left', on='CO_PAIS')

In [ ]:
df_merge3.head(2)

In [ ]:
df_tab08.duplicated(subset=['CO_UNID']).sum()

In [ ]:
chave_tab08=['CO_UNID', 'SG_UNID']

df_unids=df_tab08.groupby(chave_tab08).sum().reset_index()

In [ ]:
df_merge4=pd.merge(df_merge3, df_unids, how='left', on='CO_UNID')

In [ ]:
df_merge5=pd.merge(df_merge4, df_tab02, how='left', on='CO_NCM')

In [ ]:
df_merge6=pd.merge(df_merge5, df_tab07, how='left', on='CO_VIA')

In [ ]:
df_merge6.head(2)

In [ ]:
df_merge6=df_merge6.drop(['CO_UNID', 'CO_PAIS', 'CO_URF', 'CO_VIA','QT_ESTAT','VL_FRETE','VL_SEGURO' ], axis=1)

In [ ]:
df_merge6.insert(5,'NO_NCM_POR', df_merge6.pop('NO_NCM_POR'))
df_merge6.insert(6,'NO_ISIC_SECAO', df_merge6.pop('NO_ISIC_SECAO'))
df_merge6.insert(7,'NO_ISIC_CLASSE', df_merge6.pop('NO_ISIC_CLASSE'))
df_merge6.insert(2,'CO_SH4', df_merge6.pop('CO_SH4'))
df_merge6.insert(3,'NO_SH4_POR', df_merge6.pop('NO_SH4_POR'))
df_merge6.insert(13,'NO_PAIS', df_merge6.pop('NO_PAIS'))
df_merge6.insert(15,'SG_UNID', df_merge6.pop('SG_UNID'))
df_merge6.insert(8,'SG_UF_NCM', df_merge6.pop('SG_UF_NCM'))
df_merge6.insert(9,'NO_URF2', df_merge6.pop('NO_URF2'))
df_merge6.insert(10,'LAT', df_merge6.pop('LAT'))
df_merge6.insert(11,'LONG', df_merge6.pop('LONG'))
df_merge6.insert(14,'NO_VIA', df_merge6.pop('NO_VIA'))
df_merge6.insert(16,'KG_LIQUIDO', df_merge6.pop('KG_LIQUIDO'))

In [ ]:
df_imp_silver = df_merge6

In [ ]:
df_imp_silver['tipo'] = 'IMPORTAÇÃO'

In [ ]:
df_imp_silver

In [ ]:
df_imp_silver.info()

In [ ]:
df_imp_silver.to_csv('imp_silver_ncm.csv')

### BASE EXPORTAÇÃO

In [ ]:
#inserindo cod_sh4 pelo cod_ncm
df_exp_NCM['CO_SH4']=[numero[0:4] for numero in df_exp_NCM['CO_NCM']]

In [ ]:
df_exp_NCM.head(2)

In [ ]:
#procurando nulos no depara sh4
df_tab04=df_tab04.loc[df_tab04['CO_SH4'].notnull()]

In [ ]:
#contando numeros cod_sh4
df_tab04.duplicated(subset=['CO_SH4']).sum()

In [ ]:
df_merge1=pd.merge(df_exp_NCM, df_tab06, how='left', on='CO_URF')

In [ ]:
df_merge2=pd.merge(df_merge1, df_tab04, how='left', on='CO_SH4')

In [ ]:
df_merge2.head(2)

In [ ]:
df_merge3=pd.merge(df_merge2, df_tab03, how='left', on='CO_PAIS')

In [ ]:
df_merge3.head(2)

In [ ]:
df_tab08.duplicated(subset=['CO_UNID']).sum()

In [ ]:
chave_tab08=['CO_UNID', 'SG_UNID']

df_unids=df_tab08.groupby(chave_tab08).sum().reset_index()

In [ ]:
df_merge4=pd.merge(df_merge3, df_unids, how='left', on='CO_UNID')

In [ ]:
df_merge5=pd.merge(df_merge4, df_tab02, how='left', on='CO_NCM')

In [ ]:
df_merge6=pd.merge(df_merge5, df_tab07, how='left', on='CO_VIA')

In [ ]:
df_merge6.head(2)

In [ ]:
df_merge6=df_merge6.drop(['CO_UNID', 'CO_PAIS', 'CO_URF', 'CO_VIA','QT_ESTAT'], axis=1)

In [ ]:
df_merge6.insert(5,'NO_NCM_POR', df_merge6.pop('NO_NCM_POR'))
df_merge6.insert(6,'NO_ISIC_SECAO', df_merge6.pop('NO_ISIC_SECAO'))
df_merge6.insert(7,'NO_ISIC_CLASSE', df_merge6.pop('NO_ISIC_CLASSE'))
df_merge6.insert(2,'CO_SH4', df_merge6.pop('CO_SH4'))
df_merge6.insert(3,'NO_SH4_POR', df_merge6.pop('NO_SH4_POR'))
df_merge6.insert(13,'NO_PAIS', df_merge6.pop('NO_PAIS'))
df_merge6.insert(15,'SG_UNID', df_merge6.pop('SG_UNID'))
df_merge6.insert(8,'SG_UF_NCM', df_merge6.pop('SG_UF_NCM'))
df_merge6.insert(9,'NO_URF2', df_merge6.pop('NO_URF2'))
df_merge6.insert(10,'LAT', df_merge6.pop('LAT'))
df_merge6.insert(11,'LONG', df_merge6.pop('LONG'))
df_merge6.insert(14,'NO_VIA', df_merge6.pop('NO_VIA'))
df_merge6.insert(16,'KG_LIQUIDO', df_merge6.pop('KG_LIQUIDO'))

In [ ]:
df_exp_silver = df_merge6

In [ ]:
df_exp_silver['tipo'] = 'EXPORTAÇÃO'

In [ ]:
df_exp_silver

In [ ]:
df_exp_silver.info()

In [ ]:
df_exp_silver.to_csv('exp_silver_ncm.csv')

### Junção das Bases Imp e Exp

In [12]:
base_silver_imp =r"C:\Users\jetin\Python_Códigos\API2_fluxo_logistico\imp_silver_ncm.csv"
base_silver_exp =r"C:\Users\jetin\Python_Códigos\API2_fluxo_logistico\exp_silver_ncm.csv"

In [13]:
dict_type ={'CO_ANO' : 'category',
'CO_MES' : 'category',
'CO_NCM' : 'category',
'NO_NCM_POR' : 'category',
'NO_ISIC_SECAO' : 'category',
'NO_ISIC_CLASSE' : 'category',
'CO_SH4' : 'category',
'NO_SH4_POR' : 'category',
'SG_UNID' : 'category',
'SG_UF_NCM' : 'category',
'CO_URF' : 'category',
'NO_URF2' : 'category',
'LAT' : 'category',
'LONG' : 'category',
'NO_VIA' : 'category',
'KG_LIQUIDO' : int,
'VL_FOB' : int,
'tipo' : 'category' }

In [7]:
df_silver_imp=pd.read_csv(base_silver_imp, sep=",", dtype=dict_type, encoding='utf-8')
df_silver_exp=pd.read_csv(base_silver_exp, sep=",", dtype=dict_type, encoding='utf-8')

In [16]:
df_silver_exp

,Unnamed: 0,CO_ANO,CO_MES,CO_SH4,NO_SH4_POR,CO_NCM,NO_NCM_POR,NO_ISIC_SECAO,SG_UF_NCM,NO_URF2,LAT,LONG,NO_ISIC_CLASSE,VL_FOB,NO_VIA,NO_PAIS,SG_UNID,KG_LIQUIDO,tipo
0,0,2023,01,7009,"Espelhos de vidro, mesmo emoldurados, inclu�do...",70091000,Espelhos retrovisores para veículos,Indústria de Transformação,RS,PORTO DE SANTOS,-23.9660408,-46.3019437,Fabricação de produtos de vidro e vidro,602,MARITIMA,Peru,KGL,77,EXPORTAÇÃO
1,1,2023,01,7009,"Espelhos de vidro, mesmo emoldurados, inclu�do...",70091000,Espelhos retrovisores para veículos,Indústria de Transformação,RS,PORTO DE SANTOS,-23.9660408,-46.3019437,Fabricação de produtos de vidro e vidro,602,MARITIMA,Peru,KGL,77,EXPORTAÇÃO
2,2,2023,01,7009,"Espelhos de vidro, mesmo emoldurados, inclu�do...",70091000,Espelhos retrovisores para veículos,Indústria de Transformação,RS,PORTO DE SANTOS,-23.9660408,-46.3019437,Fabricação de produtos de vidro e vidro,602,MARITIMA,Peru,KGL,77,EXPORTAÇÃO
3,3,2023,01,8511,Aparelhos e dispositivos el�tricos de igni��o ...,85114000,"Motores de arranque, mesmo funcionando como ge...",Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,Fabricação de peças e acessórios para veículos...,13994,MARITIMA,�frica do Sul,UNID.,548,EXPORTAÇÃO
4,4,2023,01,8511,Aparelhos e dispositivos el�tricos de igni��o ...,85114000,"Motores de arranque, mesmo funcionando como ge...",Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,Fabricação de peças e acessórios para veículos...,13994,MARITIMA,�frica do Sul,UNID.,548,EXPORTAÇÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8003665,8003665,2023,05,1901,Extractos de malte; prepara��es aliment�cias d...,19012000,Misturas e pastas para a preparação de produto...,Indústria de Transformação,SC,PACARAIMA,4.478414,-61.1482639,Fabricação de produtos de moagem de grãos,3679,RODOVIARIA,Venezuela,KGL,3504,EXPORTAÇÃO
8003666,8003666,2023,05,0702,"Tomates, frescos ou refrigerados",07020000,"Tomates, frescos ou refrigerados",Agropecuária,SP,SAO SEBASTIAO,-23.8010769,-45.402614,"Cultivo de hortaliças e melões, raízes e tubér...",35,MARITIMA,Barein,KGL,23,EXPORTAÇÃO
8003667,8003667,2023,02,3208,"Tintas e vernizes, � base de pol�meros sint�ti...",32081010,"Tintas de poliésteres, dispersos/dissolv.meio ...",Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE VIRACOPOS,-23.008205,-47.13756850000001,"Fabricação de tintas, vernizes e produtos simi...",19,AEREA,Jap�o,KGL,7,EXPORTAÇÃO
8003668,8003668,2023,02,3208,"Tintas e vernizes, � base de pol�meros sint�ti...",32081010,"Tintas de poliésteres, dispersos/dissolv.meio ...",Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE VIRACOPOS,-23.008205,-47.13756850000001,"Fabricação de tintas, vernizes e produtos simi...",19,AEREA,Jap�o,KGL,7,EXPORTAÇÃO


In [17]:
df_concat_cad_prod=pd.concat([df_silver_imp,df_silver_exp])

In [18]:
df_concat_cad_prod=df_concat_cad_prod.reset_index(drop=True)

In [19]:
df_concat_cad_prod

,Unnamed: 0,CO_ANO,CO_MES,CO_SH4,NO_SH4_POR,CO_NCM,NO_NCM_POR,NO_ISIC_SECAO,SG_UF_NCM,NO_URF2,LAT,LONG,NO_ISIC_CLASSE,VL_FOB,NO_VIA,NO_PAIS,SG_UNID,KG_LIQUIDO,tipo
0,0,2023,01,8529,Partes reconhec�veis como exclusiva ou princip...,85299040,Outras partes para aparelhos de radionavegação,Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS,-23.4305731,-46.473043,Fabricação de produtos eletrônicos de consumo,968,AEREA,Hungria,KGL,1,IMPORTAÇÃO
1,1,2023,01,8529,Partes reconhec�veis como exclusiva ou princip...,85299040,Outras partes para aparelhos de radionavegação,Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS,-23.4305731,-46.473043,Fabricação de produtos eletrônicos de consumo,968,AEREA,Hungria,KGL,1,IMPORTAÇÃO
2,2,2023,01,8481,"Torneiras, v�lvulas (inclu�das as redutoras de...",84818093,Válvulas tipo gaveta,Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,"Fabricação de outras bombas, compressores, tor...",101402,MARITIMA,It�lia,UNID.,4233,IMPORTAÇÃO
3,3,2023,01,8481,"Torneiras, v�lvulas (inclu�das as redutoras de...",84818093,Válvulas tipo gaveta,Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,"Fabricação de outras bombas, compressores, tor...",101402,MARITIMA,It�lia,UNID.,4233,IMPORTAÇÃO
4,4,2023,01,8481,"Torneiras, v�lvulas (inclu�das as redutoras de...",84818093,Válvulas tipo gaveta,Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,"Fabricação de outras bombas, compressores, tor...",101402,MARITIMA,It�lia,UNID.,4233,IMPORTAÇÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18852792,8003665,2023,05,1901,Extractos de malte; prepara��es aliment�cias d...,19012000,Misturas e pastas para a preparação de produto...,Indústria de Transformação,SC,PACARAIMA,4.478414,-61.1482639,Fabricação de produtos de moagem de grãos,3679,RODOVIARIA,Venezuela,KGL,3504,EXPORTAÇÃO
18852793,8003666,2023,05,0702,"Tomates, frescos ou refrigerados",07020000,"Tomates, frescos ou refrigerados",Agropecuária,SP,SAO SEBASTIAO,-23.8010769,-45.402614,"Cultivo de hortaliças e melões, raízes e tubér...",35,MARITIMA,Barein,KGL,23,EXPORTAÇÃO
18852794,8003667,2023,02,3208,"Tintas e vernizes, � base de pol�meros sint�ti...",32081010,"Tintas de poliésteres, dispersos/dissolv.meio ...",Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE VIRACOPOS,-23.008205,-47.13756850000001,"Fabricação de tintas, vernizes e produtos simi...",19,AEREA,Jap�o,KGL,7,EXPORTAÇÃO
18852795,8003668,2023,02,3208,"Tintas e vernizes, � base de pol�meros sint�ti...",32081010,"Tintas de poliésteres, dispersos/dissolv.meio ...",Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE VIRACOPOS,-23.008205,-47.13756850000001,"Fabricação de tintas, vernizes e produtos simi...",19,AEREA,Jap�o,KGL,7,EXPORTAÇÃO


In [20]:
df_concat_cad_prod.duplicated().sum()

0

In [21]:
cadeias_produtivas = {'9401' : 'Moveleira','1517' : 'Alimentar','0402' : 'Alimentar','8438' : 'Alimentar','2715' : 'Petroleira',
'8417' : 'Metalúrgica','3907' : 'Petroquímica','2933' : 'Produtos Químicos Orgânicos','3808' : 'Agronegócio',
'2905' : 'Produtos Químicos Orgânicos','3909' : 'Petroleira','2929' : 'Produtos Químicos','2934' : 'Produtos Químicos Orgânicos',
'2931' : 'Produtos Químicos Orgânicos','6306' : 'Têxtil','9506' : 'Cultura e entretenimento','5407' : 'Têxtil',
'8526' : 'Telecomunicações','8452' : 'Têxtil','6815' : 'Minério','5903' : 'Têxtil','7601' : 'Metalúrgica',
'8708' : 'Automotiva','3402' : 'Produtos Químicos Orgânicos','7612' : 'Metalúrgica','2836' : 'Produtos Químicos',
'3105' : 'Agronegócio','3102' : 'Agronegócio','3824' : 'Metalúrgica',
'8411' : 'Aeronáutica','3002' : 'Farmacêutica','8803' : 'Aeronáutica','8807' : 'Aeronáutica','2709' : 'Petroleira',
'2710' : 'Petroleira','1003' : 'Alimentar','1107' : 'Alimentar','2833' : 'Produtos Químicos','8517' : 'Materiais Elétricos',
'8542' : 'Materiais Elétricos','7326' : 'Metalúrgica','8481' : 'Metalúrgica','7007' : 'Vidreira','8544' : 'Materiais Elétricos',
'1905' : 'Alimentar','8607' : 'Automotiva','1806' : 'Alimentar','7112' : 'Minérios','3903' : 'Petroleira',
'4703' : 'Papel e Celulose','4810' : 'Papel e Celulose','7005' : 'Vidreira','8802' : 'Aeronáutica',
'8704' : 'Automotiva','3005' : 'Farmacêutica','0202' : 'Alimentar','0306' : 'Alimentar',
'2202' : 'Alimentar','8703' : 'Automotiva','8410' : 'Metalúrgica','3302' : 'Farmacêutica','3004' : 'Farmacêutica',                     
'8457' : 'Metalúrgica', '2924' : 'Agronegócio', '8407' : 'Automotiva', '3104' : 'Agronegócio', '3304' : 'Farmacêutica',
'9032' : 'Aparelhos Eletrônicos', '8409' : 'Automotiva', '8805' : 'Aeronáutica','8529' : 'Automotiva',
'8439' : 'Papel e Celulose', '8462' : 'Metalúrgica', '9026' : 'Aparelhos Eletrônicos','2924' : 'Produtos Químicos Orgânicos',
'3815' : 'Produtos Químicos', '9031' : 'Aparelhos Eletrônicos', '7010' : 'Farmacéutica','2916' : 'Produtos Químicos Orgânicos',
'7606' : 'Automotiva', '7616' : 'Construção Civil', '8483' : 'Automotiva','8450' : 'Aparelhos Eletrônicos',   
'8415' : 'Aparelhos Eletrônicos', '0405' : 'Alimentar','0801' : 'Alimentar','0802' : 'Alimentar','0813' : 'Alimentar',
'1702' : 'Alimentar', '2008' : 'Alimentar' }

In [23]:
df_concat_cad_prod['cadeia_produtiva'] = df_concat_cad_prod['CO_SH4'].map(cadeias_produtivas)

In [24]:
df_concat_cad_prod

,Unnamed: 0,CO_ANO,CO_MES,CO_SH4,NO_SH4_POR,CO_NCM,NO_NCM_POR,NO_ISIC_SECAO,SG_UF_NCM,NO_URF2,LAT,LONG,NO_ISIC_CLASSE,VL_FOB,NO_VIA,NO_PAIS,SG_UNID,KG_LIQUIDO,tipo,cadeia_produtiva
0,0,2023,01,8529,Partes reconhec�veis como exclusiva ou princip...,85299040,Outras partes para aparelhos de radionavegação,Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS,-23.4305731,-46.473043,Fabricação de produtos eletrônicos de consumo,968,AEREA,Hungria,KGL,1,IMPORTAÇÃO,Automotiva
1,1,2023,01,8529,Partes reconhec�veis como exclusiva ou princip...,85299040,Outras partes para aparelhos de radionavegação,Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS,-23.4305731,-46.473043,Fabricação de produtos eletrônicos de consumo,968,AEREA,Hungria,KGL,1,IMPORTAÇÃO,Automotiva
2,2,2023,01,8481,"Torneiras, v�lvulas (inclu�das as redutoras de...",84818093,Válvulas tipo gaveta,Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,"Fabricação de outras bombas, compressores, tor...",101402,MARITIMA,It�lia,UNID.,4233,IMPORTAÇÃO,Metalúrgica
3,3,2023,01,8481,"Torneiras, v�lvulas (inclu�das as redutoras de...",84818093,Válvulas tipo gaveta,Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,"Fabricação de outras bombas, compressores, tor...",101402,MARITIMA,It�lia,UNID.,4233,IMPORTAÇÃO,Metalúrgica
4,4,2023,01,8481,"Torneiras, v�lvulas (inclu�das as redutoras de...",84818093,Válvulas tipo gaveta,Indústria de Transformação,SP,PORTO DE SANTOS,-23.9660408,-46.3019437,"Fabricação de outras bombas, compressores, tor...",101402,MARITIMA,It�lia,UNID.,4233,IMPORTAÇÃO,Metalúrgica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18852792,8003665,2023,05,1901,Extractos de malte; prepara��es aliment�cias d...,19012000,Misturas e pastas para a preparação de produto...,Indústria de Transformação,SC,PACARAIMA,4.478414,-61.1482639,Fabricação de produtos de moagem de grãos,3679,RODOVIARIA,Venezuela,KGL,3504,EXPORTAÇÃO,NaN
18852793,8003666,2023,05,0702,"Tomates, frescos ou refrigerados",07020000,"Tomates, frescos ou refrigerados",Agropecuária,SP,SAO SEBASTIAO,-23.8010769,-45.402614,"Cultivo de hortaliças e melões, raízes e tubér...",35,MARITIMA,Barein,KGL,23,EXPORTAÇÃO,NaN
18852794,8003667,2023,02,3208,"Tintas e vernizes, � base de pol�meros sint�ti...",32081010,"Tintas de poliésteres, dispersos/dissolv.meio ...",Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE VIRACOPOS,-23.008205,-47.13756850000001,"Fabricação de tintas, vernizes e produtos simi...",19,AEREA,Jap�o,KGL,7,EXPORTAÇÃO,NaN
18852795,8003668,2023,02,3208,"Tintas e vernizes, � base de pol�meros sint�ti...",32081010,"Tintas de poliésteres, dispersos/dissolv.meio ...",Indústria de Transformação,SP,AEROPORTO INTERNACIONAL DE VIRACOPOS,-23.008205,-47.13756850000001,"Fabricação de tintas, vernizes e produtos simi...",19,AEREA,Jap�o,KGL,7,EXPORTAÇÃO,NaN


In [25]:
df_concat_cad_prod['cadeia_produtiva'] = df_concat_cad_prod['cadeia_produtiva'].fillna('Outras')

In [26]:
df_concat_cad_prod['cadeia_produtiva'].isna().sum()

0

In [27]:
df_concat_cad_prod['cadeia_produtiva'].value_counts()

Outras                         14238908
Automotiva                      1460857
Materiais Elétricos              722639
Metalúrgica                      494406
Produtos Químicos Orgânicos      422343
Alimentar                        339019
Aparelhos Eletrônicos            261667
Farmacêutica                     219086
Moveleira                        155001
Têxtil                            94003
Agronegócio                       87822
Petroleira                        76899
Petroquímica                      68796
Cultura e entretenimento          49266
Produtos Químicos                 36476
Papel e Celulose                  31614
Aeronáutica                       23246
Construção Civil                  18150
Minério                           17696
Vidreira                          15264
Telecomunicações                   9027
Farmacéutica                       8561
Minérios                           2051
Name: cadeia_produtiva, dtype: int64

In [30]:
df_concat_cad_prod['CO_ANO']=df_concat_cad_prod['CO_ANO'].astype(str)
df_concat_cad_prod['CO_MES']=df_concat_cad_prod['CO_MES'].astype(str)

In [31]:
df_concat_cad_prod['dia']='01'

df_concat_cad_prod['data']=df_concat_cad_prod['CO_ANO']+'-'+df_concat_cad_prod['CO_MES']+'-'+df_concat_cad_prod['dia']

df_concat_cad_prod['data']=pd.to_datetime(df_concat_cad_prod['data'], format='%Y-%m-%d')

In [ ]:
df_concat_cad_prod['cod_ncm_nome']=df_concat_cad_prod['CO_NCM']+'-'+df_concat_cad_prod['NO_NCM_POR']

In [ ]:
df_concat_cad_prod

In [ ]:
df_concat_cad_prod.to_csv('df_concat_elo_3.csv', index=False)

In [ ]:
dict_csv ={'CO_ANO' : 'category',
'CO_MES' : 'category',
'CO_NCM' : 'category',
'NO_NCM_POR' : 'category',
'NO_ISIC_SECAO' : 'category',
'NO_ISIC_CLASSE' : 'category',
'NO_CUCI_GRUPO' : 'category',
'CO_SH4' : 'category',
'NO_SH4_POR' : 'category',
'CO_SH2' : 'category',
'NO_SH2_POR' : 'category',
'CO_PAIS' : 'category',
'NO_PAIS' : 'category',
'CO_UNID' : 'category',
'SG_UNID' : 'category',
'SG_UF_NCM' : 'category',
'CO_URF' : 'category',
'NO_URF2' : 'category',
'LAT' : 'category',
'LONG' : 'category',
'CO_VIA' : 'category',
'NO_VIA' : 'category',
'KG_LIQUIDO' : int,
'QT_ESTAT' : int,
'VL_FOB' : int,
'elos_cadeia_produtiva' : 'category',
'tipo' : 'category',
'cadeia_produtiva' : 'category',
'data' : 'category',
'cod_sh4_nome_sh2' : 'category' }

In [ ]:
df_elo=pd.read_csv(base_elo, sep=",", dtype=dict_csv)

In [ ]:
df_elo.info()

In [ ]:
df_elo=df_elo.drop(['NO_ISIC_SECAO', 'NO_CUCI_GRUPO', 'NO_SH4_POR', 'CO_SH2','NO_SH2_POR', 'CO_PAIS', 'CO_UNID', 'CO_URF', 'CO_VIA' ], axis=1)